In [ ]:
#read csv data from archive (1)


In [20]:
import os
import pandas as pd

print(os.getcwd())

df = pd.read_csv('../data/archive/financial_news_events.csv')

nifty_df = df[df['Market_Index'] == 'NSE Nifty'][['Date','Headline']]

nifty_df.head()

d:\2023AC05493-Project\notebooks


,Date,Headline
3,2025-07-21,Massive stock buyback program announced by a c...
21,2025-08-03,Mining company stock surges on commodity price...
37,2025-06-13,Bond yields tumble as investors seek safe havens
51,2025-02-09,Massive stock buyback program announced by a c...
164,2025-04-16,Central bank maintains status quo on interest ...


In [28]:
df = pd.read_csv("..//data//archive (1)//NifSent//final_news_sentiment_analysis.csv")

nifty_df_1 = df[['Publish Date','Headline']]

nifty_df_1.head()

#Filter only date greater than 2024-12-31
nifty_df_1 = nifty_df_1[nifty_df_1['Publish Date'] > '2024-12-31']

nifty_df_1.head()


,Publish Date,Headline
1644,2025-01-09,Apollo Hospitals receives Investment Bank Anal...
1808,2025-01-09,Apollo Hospitals receives Investment Bank Anal...
2827,2025-01-07,Bajaj Finserv Limited receives Investment Bank...
2828,2025-01-02,Kotak reinstates Bajaj Finserv stock with Buy ...
2829,2025-01-02,Bajaj Finserv Limited receives Investment Bank...


In [27]:
df = pd.read_csv("..//data//archive (1)//NifSent//Data Extraction//WebScrapping//merged_scrapping.csv")

nifty_df_2 = df[['Date','Headline']]

nifty_df_2.head()

,Date,Headline
0,2026-01-27,Adani Group and Embraer partner to build regio...
1,2026-01-23,Adani group firms shed $12.5 billion market ca...
2,2026-01-23,Adani shares tumble as U.S. regulator seeks to...
3,2025-12-16,Exclusive-Top Indian arms makers held rare mee...
4,2025-11-28,Adani plans $5 billion investment in Google da...


In [30]:
a = nifty_df[['Date', 'Headline']].copy()
b = nifty_df_1.rename(columns={'Publish Date': 'Date'})[['Date', 'Headline']].copy()
c = nifty_df_2[['Date', 'Headline']].copy()

In [32]:
all_df = pd.concat([a, b, c], ignore_index=True)

# Parse dates (normalize to midnight) and drop invalid rows
all_df['Date'] = pd.to_datetime(all_df['Date'], errors='coerce').dt.normalize()
all_df = all_df.dropna(subset=['Date'])

# Sort so we keep the most recent articles when taking up to 5 per date
all_df = all_df.sort_values('Date', ascending=False)

# Group by date and aggregate headlines into lists (limit to 5)
grouped = (all_df.groupby('Date', sort=True)['Headline']
           .apply(lambda s: list(s)[:5])
           .to_frame(name='Headlines'))

# Reindex to full daily range 2020-01-01 through 2025-12-31 and backfill missing dates
full_index = pd.date_range('2020-01-01', '2025-12-31', freq='D')
grouped = grouped.reindex(full_index)

# Backfill (use next available day's headlines) and then forward-fill any remaining gaps
grouped['Headlines'] = grouped['Headlines'].bfill().ffill()

# Reset index and name column
grouped = grouped.reset_index().rename(columns={'index': 'Date'})

# Final consolidated dataframe
nifty_union = grouped

# Quick check
nifty_union

,Date,Headlines
0,2020-01-01,"[Tata Consumer reports moderate India growth, ..."
1,2020-01-02,"[Tata Consumer reports moderate India growth, ..."
2,2020-01-03,"[Tata Consumer reports moderate India growth, ..."
3,2020-01-04,"[Tata Consumer reports moderate India growth, ..."
4,2020-01-05,"[Tata Consumer reports moderate India growth, ..."
...,...,...
2187,2025-12-27,"[Form 6K HDFC BANK LTD For: 29 December, Form ..."
2188,2025-12-28,"[Form 6K HDFC BANK LTD For: 29 December, Form ..."
2189,2025-12-29,"[Form 6K HDFC BANK LTD For: 29 December, Form ..."
2190,2025-12-30,[Shriram Finance Ltd receives Investment Bank ...


In [34]:
nifty_union.to_csv("../data/consolidated_nifty_news.csv", index=False)